<a href="https://www.kaggle.com/code/priyankapalshetkar/spaceship-titanic-score-0-8?scriptVersionId=117912606" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

clear_output()

## Imports and Setup

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
# from lazypredict.Supervised import LazyClassifier

In [4]:
SEED = 42

def seed_everything(seed = 42):
    import random, os
    import numpy as np

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

In [5]:
train_df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
sample_submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

## Data Fields Information:
* PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
* HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
* CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
* Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
* Destination - The planet the passenger will be debarking to.
* Age - The age of the passenger.
* VIP - Whether the passenger has paid for special VIP service during the voyage.
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
* Name - The first and last names of the passenger.
* Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

## Data Study

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [7]:
train_df.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


### Some Prior Observations/points to explore:
* RoomService, FoodCourt, ShoppingMall, Spa, VRDeck are related to CryoSleep because if the passenger has elected to be put in suspended animation, they would not spend money on these amenities.
* CryoSleep and Cabin could be related i.e. passengers in CryoSleep could be in a particular section of the ship?
* VIP and CryoSleep could also be related the same way as RoomService, FoodCourt, ShoppingMall, Spa, VRDeck because why would a passenger pay for special VIP service if they are going to be in CryoSleep?
* HomePlanet and Destination might also be related as people from a particular home planet might plan to go to the same destination?

### Studying the Target variable

In [8]:
train_df["Transported"].astype(int).describe()

count    8693.000000
mean        0.503624
std         0.500016
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Transported, dtype: float64

## Missing Values

In [9]:
print(train_df.isnull().sum().sort_values(ascending=False))
print(test_df.isnull().sum().sort_values(ascending=False))

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64
FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
VRDeck           80
PassengerId       0
dtype: int64


In [10]:
def wrangle(df):
    
    # Split Cabin column into deck/num/side
    df[["Cabin_deck", "Cabin_num", "Cabin_side"]] = df["Cabin"].str.split("/", expand = True)
    
    # Create a new column Passenger Group from PassengerId
    df["Passenger_Group"] = df["PassengerId"].str.split("_", expand=True)[0].astype(int)
    
    # Drop name column
    df.drop(columns="Name", inplace=True)
    
    # Fill in missing values
    
    df["CryoSleep"].fillna("False", inplace = True)
    df["VIP"].fillna("False", inplace = True)
    df["CryoSleep"] = df["CryoSleep"].astype(bool)
    df["VIP"] = df["VIP"].astype(bool)
    df["HomePlanet"].fillna("Earth", inplace = True)
    df["Destination"].fillna("TRAPPIST-1e", inplace = True)
    df["Age"].fillna(df["Age"].median(), inplace=True)
    
    for col in ["Cabin_deck", "Cabin_side"]:
        df[col].fillna("Unknown", inplace = True)
        
    for col in ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]:
        df.loc[df[col].isna() & df["CryoSleep"] == True, col] = 0
        df[col].fillna(df[col].median(), inplace=True)
    
    # Create a new column Spending Level from RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
    total_spending = df["RoomService"] + df["FoodCourt"] + df["ShoppingMall"] + df["Spa"] + df["VRDeck"]
    df["Spending_Level"] = pd.cut(x=total_spending, bins=[-1,0,2500,5000,100000], labels=['None','Low','Moderate','High'])
    
    # Dropping not needed columns
    df.drop(columns=["Cabin", "Cabin_num"], inplace = True)
    
    return df

In [11]:
train_df = wrangle(train_df)
test_df = wrangle(test_df)

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   PassengerId      8693 non-null   object  
 1   HomePlanet       8693 non-null   object  
 2   CryoSleep        8693 non-null   bool    
 3   Destination      8693 non-null   object  
 4   Age              8693 non-null   float64 
 5   VIP              8693 non-null   bool    
 6   RoomService      8693 non-null   float64 
 7   FoodCourt        8693 non-null   float64 
 8   ShoppingMall     8693 non-null   float64 
 9   Spa              8693 non-null   float64 
 10  VRDeck           8693 non-null   float64 
 11  Transported      8693 non-null   bool    
 12  Cabin_deck       8693 non-null   object  
 13  Cabin_side       8693 non-null   object  
 14  Passenger_Group  8693 non-null   int64   
 15  Spending_Level   8693 non-null   category
dtypes: bool(3), category(1), float64(6), int64

In [13]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   PassengerId      4277 non-null   object  
 1   HomePlanet       4277 non-null   object  
 2   CryoSleep        4277 non-null   bool    
 3   Destination      4277 non-null   object  
 4   Age              4277 non-null   float64 
 5   VIP              4277 non-null   bool    
 6   RoomService      4277 non-null   float64 
 7   FoodCourt        4277 non-null   float64 
 8   ShoppingMall     4277 non-null   float64 
 9   Spa              4277 non-null   float64 
 10  VRDeck           4277 non-null   float64 
 11  Cabin_deck       4277 non-null   object  
 12  Cabin_side       4277 non-null   object  
 13  Passenger_Group  4277 non-null   int64   
 14  Spending_Level   4277 non-null   category
dtypes: bool(2), category(1), float64(6), int64(1), object(5)
memory usage: 413.8+ KB


## Encoding of Categorical Variables

In [14]:
columns = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP','Cabin_deck', 'Cabin_side', 'Spending_Level']
# train_df = get_one_hot_encoded_data(train_df, columns)
# test_df = get_one_hot_encoded_data(test_df, columns)

In [15]:
train_df = pd.get_dummies(train_df, columns=columns)
test_df = pd.get_dummies(test_df, columns=columns)

In [16]:
# def label_encoder(df):
#     for col in columns:
#         df[col] = df[col].astype(str)
#         df[col] =  LabelEncoder().fit_transform(df[col])
#     return df

In [17]:
# train_df = label_encoder(train_df)
# test_df = label_encoder(test_df)

In [18]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   Age                        8693 non-null   float64
 2   RoomService                8693 non-null   float64
 3   FoodCourt                  8693 non-null   float64
 4   ShoppingMall               8693 non-null   float64
 5   Spa                        8693 non-null   float64
 6   VRDeck                     8693 non-null   float64
 7   Transported                8693 non-null   bool   
 8   Passenger_Group            8693 non-null   int64  
 9   HomePlanet_Earth           8693 non-null   uint8  
 10  HomePlanet_Europa          8693 non-null   uint8  
 11  HomePlanet_Mars            8693 non-null   uint8  
 12  CryoSleep_False            8693 non-null   uint8  
 13  CryoSleep_True             8693 non-null   uint8

## Train-Test Split

In [19]:
X = train_df.drop(columns=['PassengerId','Transported'])
print(X.shape)
y = train_df.Transported
print(y.shape)

(8693, 33)
(8693,)


In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,
                                                        test_size=0.1, 
                                                        stratify=y, 
                                                        random_state=SEED)

## Finding the best base Model using Lazy Predict

In [21]:
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(X_train, X_valid, y_train, y_valid)
# models

## Model Training

In [22]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier()

## Inference

In [23]:
XTest = test_df.drop(columns=['PassengerId'])

In [24]:
XTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        4277 non-null   float64
 1   RoomService                4277 non-null   float64
 2   FoodCourt                  4277 non-null   float64
 3   ShoppingMall               4277 non-null   float64
 4   Spa                        4277 non-null   float64
 5   VRDeck                     4277 non-null   float64
 6   Passenger_Group            4277 non-null   int64  
 7   HomePlanet_Earth           4277 non-null   uint8  
 8   HomePlanet_Europa          4277 non-null   uint8  
 9   HomePlanet_Mars            4277 non-null   uint8  
 10  CryoSleep_False            4277 non-null   uint8  
 11  CryoSleep_True             4277 non-null   uint8  
 12  Destination_55 Cancri e    4277 non-null   uint8  
 13  Destination_PSO J318.5-22  4277 non-null   uint8

In [25]:
# ytest_pred = trained_models['random_forest']['model'].predict(XTest)
ytest_pred = clf.predict(XTest)

In [26]:
output = sample_submission.copy()
output['Transported'] = ytest_pred
output.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [27]:
output.to_csv('./solution.csv', index=False)